In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 6.9 MB/s 
     |████████████████████████████████| 132 kB 22.7 MB/s 
     |████████████████████████████████| 212 kB 33.4 MB/s 
     |████████████████████████████████| 182 kB 15.9 MB/s 
     |████████████████████████████████| 127 kB 21.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 7.3 MB/s 
     |████████████████████████████████| 132 kB 54.6 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
     |████████████████████████████████| 10.2 MB 52.6 MB/s 


In [ ]:
from datasets import list_datasets, load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pprint import pprint

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
from pprint import pprint

In [ ]:
from google.colab import files

In [ ]:
# files.upload()

In [ ]:
dataframe = pd.read_csv('first100.csv', index_col='Unnamed: 0')

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   article        97 non-null     object
 1   abstract       100 non-null    object
 2   section_names  100 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


In [ ]:
dataframe.head(n=1)

,article,abstract,section_names
0,a recent systematic analysis showed that in 20...,background : the present study was carried ou...,INTRODUCTION\nMATERIALS AND METHODS\nParticipa...


In [ ]:
from typing import List

In [ ]:
import string

In [ ]:
from pytorch_pretrained_bert import BertModel, GPT2Model, GPT2Tokenizer, BertTokenizer

In [ ]:
import torch

In [ ]:
from tqdm import tqdm

In [ ]:
def process_text(body: str) -> List[str]:
    sentences = body.split(".")

    # converting data into lower case
    sentences = [sentence.lower() for sentence in sentences]

    # Removing the punctuations
    punctuations = str.maketrans('', '', string.punctuation)
    sentences = [word.translate(punctuations) for word in sentences]

    # Some Basic Cleaning
    return [sentence
            for sentence in sentences
            if len(sentence) > 75 and
            not sentence.startswith('but') and
            not sentence.startswith('and') and
            not sentence.startswith('or')]	

In [ ]:
model_handler = {
    'bert': BertModel,
    'openApi': GPT2Model
}

token_handler = {
    'openApi': GPT2Tokenizer,
    'bert': BertTokenizer
}

size_handler = {
    'base': {
        'bert': 'bert-base-uncased',
        'openApi': 'gpt2'
    },
    'large': {
        'bert': 'bert-large-uncased',
        'openApi': 'gpt2'
    }
}

vector_handler = {
    'base': {
        'bert': 768,
        'openApi': 768
    },
    'large': {
        'bert': 1024,
        'openApi': 768
    }
}

In [ ]:
# Building the Model using BERT Encoder

In [ ]:
model = model_handler['bert'].from_pretrained(size_handler['large']['bert'])

100%|██████████| 1248501532/1248501532 [00:25<00:00, 49821465.33B/s]


In [ ]:
tokenizer = token_handler['bert'].from_pretrained(size_handler['large']['bert'])

100%|██████████| 231508/231508 [00:00<00:00, 3870250.67B/s]


In [ ]:
vectorizer = vector_handler['large']['bert']

In [ ]:
print('Tokenizer has been taken for encoding the data = ', tokenizer)

Tokenizer has been taken for encoding the data =  <pytorch_pretrained_bert.tokenization.BertTokenizer object at 0x7fd5dc0337d0>


In [ ]:
print('Vectorizer has been taken for encoding the data = ', vectorizer)

Vectorizer has been taken for encoding the data =  1024


In [ ]:
def tokenize_input(text: str) -> torch.tensor:

    # Using BERT Tokenizer and tokenizing the Input
    tokens = tokenizer.tokenize(text)

    # Converting the Tokens into Ids
    indexed = tokenizer.convert_tokens_to_ids(tokens)

    # Converting the Tokens into PyTorch Tensor
    return torch.tensor([indexed])

In [ ]:
# Creating Embedding matrix for for fetching the data using below function

In [ ]:
def extract_embedding(text: str, use_hidden: bool = True, squeeze: bool = False) -> np.ndarray:

    # Tokenizing the given input data
    tokens = tokenize_input(text)

    # Getting the Hidden State using the above Tokens
    hidden_state, pooled = model(tokens)

    if use_hidden:
        pooled = hidden_state[-2].mean(dim=1)

    if squeeze:
        return pooled.detach().numpy().squeeze()

    return pooled

In [ ]:
def create_matrix(articles: List[str], use_hidden: bool = True) -> np.ndarray:

    vector = np.zeros((len(articles), vectorizer))

    for idx, text in tqdm(enumerate(articles)):

        vector[idx] = extract_embedding(text=text, use_hidden=use_hidden).data.numpy()

    return vector

##### Performing Extractive Text Summarization on the above preprocessed Sample

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
from sklearn.cluster import AffinityPropagation, KMeans

In [ ]:
# Method to perfom extractive text summarization on the given features

In [ ]:
# Performing the extractive text summarization the entire dataset for 1% ratio of the data

In [ ]:
def get_model(clusters, algorithm: str = 'kmeans'):

    if algorithm == 'gmm':
        return GaussianMixture(n_components=clusters)
    elif algorithm == 'affinity':
        return AffinityPropagation()
    else:
        return KMeans(n_clusters=clusters)


def get_centroids(model, algorithm: str = 'kmeans'):
    if algorithm == 'gmm':
        return model.means_
    else:
        return model.cluster_centers_


def find_closest_args(features, centroids):
    centroid_mean = 1e7
    current_arg = -1
    args = {}
    used_idx = []

    for jdx, centroid in enumerate(centroids):
        for idx, feature in enumerate(features):
            value = sum(abs(feature - centroid))
            if value < centroid_mean and idx not in used_idx:
                current_arg = idx
                centroid_mean = value

        used_idx.append(current_arg)
        args[jdx] = current_arg
        centroid_mean = 1e7
        current_arg = current_arg - 1
    return args


def perform_summarization(data, ratio: float = 0.01, algorithm: str = 'kmeans', pca_components: int = None):

    if pd.isna(data) or pd.isnull(data):
        return ""

    # Taking the embeddings from the dataset
    # Performing the Preprocessing of the Given Article
    preprocessed_article = process_text(body=data)

    # Extracting the embedding for the cluster
    hidden = create_matrix(articles=preprocessed_article)

    # Performing PCA decomposition on the given data 
    pca = PCA(n_components=pca_components)
    pca.fit(X=hidden)
    features = pca.transform(X=hidden) # Features generated

    # Performing the Compution using the above features generated
    clusters = 1 if ratio * len(features) < 1 else int(len(features) * ratio)

    # Getting the Given Model and fitting the same on the given features
    model = get_model(clusters=clusters)
    model.fit(X=features, y=None)

    # Getting the Centroids from the Model
    centroids = get_centroids(model=model)

    # Getting the Cluster Arguments
    cluster_args = find_closest_args(features, centroids=centroids)

    # Sorting the data and returning the same
    answers = sorted(cluster_args.values())
    if answers[0] != 0:
        answers.insert(0, 0)

    return ' '.join([preprocessed_article[arg] for arg in answers]).strip()

In [ ]:
# sample_article=dataframe.iloc[0, 0]

In [ ]:
# perform_summarization(data=sample_article)

In [ ]:
dataframe['extracted'] = dataframe.apply(lambda article: perform_summarization(article['article']), axis=1)

159it [01:37,  1.63it/s]
100it [00:59,  1.68it/s]
40it [00:24,  1.61it/s]
202it [02:05,  1.60it/s]
39it [00:23,  1.66it/s]
79it [00:52,  1.50it/s]
47it [00:33,  1.39it/s]
84it [00:51,  1.63it/s]
87it [00:52,  1.67it/s]
41it [00:27,  1.51it/s]
117it [01:09,  1.67it/s]
140it [01:29,  1.57it/s]
218it [02:34,  1.41it/s]
27it [00:14,  1.91it/s]
82it [00:40,  2.00it/s]
52it [00:39,  1.30it/s]
42it [00:28,  1.46it/s]
140it [01:20,  1.75it/s]
79it [00:36,  2.18it/s]
65it [00:49,  1.30it/s]
61it [00:34,  1.75it/s]
104it [01:14,  1.39it/s]
238it [02:26,  1.62it/s]
84it [00:44,  1.90it/s]
192it [01:54,  1.67it/s]
32it [00:17,  1.80it/s]
91it [00:56,  1.60it/s]
81it [00:43,  1.88it/s]
32it [00:17,  1.83it/s]
81it [00:41,  1.96it/s]
87it [00:51,  1.69it/s]
147it [01:23,  1.77it/s]
77it [00:47,  1.63it/s]
69it [00:38,  1.80it/s]
163it [01:36,  1.69it/s]
59it [00:32,  1.82it/s]
118it [01:09,  1.70it/s]
67it [00:38,  1.76it/s]
104it [00:53,  1.93it/s]
127it [01:22,  1.55it/s]
69it [00:35,  1.92it/s]
6

In [ ]:
dataframe[['extracted', 'abstract']].to_csv('extracted.csv', index=False)

In [ ]:
!ls -lrt

total 1944
drwxr-xr-x 1 root root    4096 Sep  9 13:22 sample_data
-rw-r--r-- 1 root root 1809500 Sep 14 10:52 first100.csv
-rw-r--r-- 1 root root  174027 Sep 14 12:50 extracted.csv


In [ ]:
!du -sh *

172K	extracted.csv
1.8M	first100.csv
55M	sample_data


In [ ]:
from google.colab import files

In [ ]:
files.download('extracted.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>